In [1]:
import subprocess

def extract_audio_from_video(video_path, audio_path):
    command = [
        "ffmpeg", 
        "-i", 
        video_path, 
        "-vn", 
        audio_path,
        ]
    subprocess.run(command)

extract_audio_from_video(
    "/Users/jihoonchoi/Projects/GPTs/files/example.mp4", 
    "/Users/jihoonchoi/Projects/GPTs/files/example.mp3"
    )

ffmpeg version 6.1.1 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.3.9.4)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/6.1.1_7 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopenvi

In [2]:
from pydub import AudioSegment
import math

track = AudioSegment.from_mp3("/Users/jihoonchoi/Projects/GPTs/files/example.mp3")

five_min = 5 * 60 * 1000
ten_min = 10 * 60 * 1000

chunks = math.ceil(len(track) / ten_min)

for i in range(chunks):
    start_time = i * ten_min
    end_time = (i+1) * ten_min

    chunk = track[start_time:end_time]

    chunk.export(f"/Users/jihoonchoi/Projects/GPTs/files/chunks/chunk_{i}.mp3")

In [19]:
def cut_audio_in_chunk(audio_path, chunk_size, chunk_folder):
    track = AudioSegment.from_mp3(audio_path)
    chunk_len = chunk_size * 60 * 1000
    chunks = math.ceil(len(track) / chunk_len)

    for i in range(chunks):
        start_time = i * ten_min
        end_time = (i+1) * ten_min

        chunk = track[start_time:end_time]

        chunk.export(f"{chunk_folder}/chunk_{i}.mp3", format="mp3")

In [21]:
cut_audio_in_chunk("/Users/jihoonchoi/Projects/GPTs/files/example.mp3", 8, "/Users/jihoonchoi/Projects/GPTs/files/chunks")

In [23]:
import openai

transcript = openai.Audio.transcribe(
    "whisper-1",
    open("/Users/jihoonchoi/Projects/GPTs/files/chunks/chunk_0.mp3", "rb")
    )

transcript['text']

{
  "text": "\uc790 \uc81c\uac00 \uc2e4\uc81c\ub85c \uacaa\uc740 \uc77c\uc740 \uc544\ub2c8\uc9c0\ub9cc \ub9c8\uce58 \uadf8\ub7f0 \uac83\ucc98\ub7fc \uc5b4\ub5a4 \uc9e7\uc740 \uc774\uc57c\uae30\ub97c \uc6b0\uc120 \ud55c\uad6d\uc5b4\ub85c \uba3c\uc800 \uc880 \ub4e4\ub824 \ub4dc\ub9b4\uac8c\uc694 \uc77c\ub2e8 \uadf8\ub0e5 \ud3b8\ud558\uac8c \ub4e4\uc5b4\ubcf4\uc138\uc694 \uce5c\uad6c\ub791 \ub531 \ub4e4\uc5b4\uac14\ub294\ub370 \uc0ac\ub78c\uc774 \uc5c4\uccad \ub9ce\ub354\ub77c\uad6c\uc694 \ube48 \ud14c\uc774\ube14 \uc5c6\uace0 \ubcf4\ub2c8\uae4c \uae08\ubc29 \uc790\ub9ac\uac00 \ub098\uc9c0\ub3c4 \uc54a\uaca0\ub354\ub77c\uad6c\uc694 \uadf8\ub798\uc11c \uce5c\uad6c \ubcf4\uace0 \ub2e4\ub978 \ub370 \uac00\uc790\uace0 \uc598\uae30\ub97c \ud558\ub824\ub294\ub370 \uc598\uac00 \uc5b4\ub290 \ud14c\uc774\ube14\uc5d0 \uc549\uc544\uc788\ub294 \ub098\uc774\uac00 \uc880 \uc788\ub294 \ucee4\ud50c\ud55c\ud14c \uac00\ub354\ub2c8 \uac19\uc774 \uc880 \uc549\uc544\ub3c4 \ub418\ub0d0\ub294 \uac70\uc5d0\uc694

In [26]:
import glob

def transcribe_chunks(chunk_folder, destination):
    files = glob.glob(f"{chunk_folder}/*.mp3")

    for file in files:
        with open(file, "rb") as audio_file, open(destination, "a") as text_file:
            transcript = openai.Audio.transcribe(
                "whisper-1",
                audio_file,
            )
            text_file.write(transcript["text"])

transcribe_chunks("/Users/jihoonchoi/Projects/GPTs/files/chunks", "/Users/jihoonchoi/Projects/GPTs/files/chunks/transcript.txt")